In [ ]:
from utils.randomstructure import generate_random_prob
from utils.polycasebuilder import generate_random_polynomials
from graph.graphutils import edges_to_Ein_Eout, sources
import numpy as np

In [2]:
def generate_individual_problem(n_eqs, n_vars, sparsity, seed1, seed2):
    eqv, varinc, dout = generate_random_prob(n_eqs, n_vars, seed1, sparsity)
    xval = np.ones(n_vars)
    polynomials, var_mapping, edges, tree = generate_random_polynomials(eqv, dout, n_eqs, seed=seed2, xval=xval)
    symb_mapping = {key: elt[0] for key, elt in var_mapping.items()}
    inputids = sorted(sources(*edges_to_Ein_Eout(edges)))
    inputvars = [var_mapping[elt][0] for elt in inputids]
    fobj = sum([(elt-1)**2 for elt in inputvars])
    
    # build string object
    functional_sets = []
    for key,expr in polynomials.items():
        output = edges[1][key][0]
        functional_sets.append(
            {'residual':str(expr), 
             'functionalvar': var_mapping[output][1]})
    output = {'functional_sets':functional_sets, 
              'equality_constraints': [], 
              'inequality_constraints':[], 
              'objective': str(fobj)}
    return output

# Generate one problem

In [3]:
problem_params = (10, 15, 1, 4, 2) #10, 13, 1.1, 42, 12345 #1.7
json_data = dict()
json_data[str(problem_params)] = generate_individual_problem(*problem_params)

In [4]:
json_data

{'(10, 15, 1, 4, 2)': {'functional_sets': [{'residual': '-0.7*x_1*x_2*x_7 + 0.2*x_10*x_6 + 0.5*x_5',
    'functionalvar': 'x_2'},
   {'residual': '-0.5*x_0*x_9 + 0.5', 'functionalvar': 'x_0'},
   {'residual': '-0.7*x_11*x_3 + 0.7*x_13*x_9', 'functionalvar': 'x_9'},
   {'residual': '-0.7*x_10 - 0.2*x_3*x_5*x_6 + 0.9*x_7*x_8',
    'functionalvar': 'x_6'},
   {'residual': '-0.3*x_10*x_13 + 0.3', 'functionalvar': 'x_13'},
   {'residual': '0.6*x_10*x_4*x_5 - 0.6', 'functionalvar': 'x_10'},
   {'residual': '0.1*x_0*x_14 - 0.1*x_11*x_8*x_9', 'functionalvar': 'x_8'},
   {'residual': '0.8*x_13*x_3*x_5 - 0.8', 'functionalvar': 'x_3'},
   {'residual': '0.4*x_7*x_9 - 0.4', 'functionalvar': 'x_7'},
   {'residual': '0.2*x_11*x_13 + 0.7*x_12*x_6 - 0.9*x_7',
    'functionalvar': 'x_11'}],
  'equality_constraints': [],
  'inequality_constraints': [],
  'objective': '(x_1 - 1)**2 + (x_12 - 1)**2 + (x_14 - 1)**2 + (x_4 - 1)**2 + (x_5 - 1)**2'}}

# Generate combinatorially many problems

In [5]:
from itertools import product
import pandas as pd
def generate_all_problems(*args, df=None):
    data = []
    colnames = ["m", "ncoeff", "n", "sparsity", "seed", "seedpoly"] 
    for m,ncoeff,sparsity,seed1, seed2 in product(*args):
        n=m+int(ncoeff*m)
        if df is None or ((m, ncoeff, n, sparsity, seed1, seed2) not in df[colnames].itertuples(index=False)):
            data.append([m, ncoeff, n, sparsity, seed1, seed2])
    df2 = pd.DataFrame(data, columns=colnames)
    if df is not None:
        df2 = pd.concat([df, df2], ignore_index=True)
    return df2

In [10]:
df = generate_all_problems((10,),(0.5,),(1,),range(10),range(10))

In [11]:
json_data = dict()
for row in df.itertuples():
    problem_id = (row.m, row.n, row.sparsity, row.seed, row.seedpoly)
    json_data[str(problem_id)] = generate_individual_problem(*problem_id)

In [12]:
# save json data
import json
with open('../applications/data/polyproblems_v2_med.json', 'w') as f:
    json.dump(json_data, f, indent=4)